In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
#set path for my_tools script
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import file_dic, plot_dic


In [6]:
#set variable to plot
variable = 'phasespeed'
#Change fontsize to make graph better readable
plt.rcParams.update({'font.size': 14})
#list all filez
filez = sorted(glob('/media/onno/Algemeen/Thesis/GFS_mean/GFS_mean_error_{}_day_??_globe_mars.nc'.format(variable)))
#Put all lead day errors into one dataset
datasets = []
for i, file in enumerate(filez):
    ds_single = xr.open_dataset(file,decode_times=False).squeeze()
    datasets.append(ds_single)
ds = xr.concat(datasets,dim='lead')
#set coordinates for either Northern Hemisphere or Souther Hemisphere. Latitudes outside 25-75 degree bound are thought of to be irrelevant
latz_hem = [(75,25)]
# latz_hem = [(-25,-75)]
leads = np.linspace(0,240,11)
for lat_hem in latz_hem:
    ds_hem = ds.sel(lat=slice(lat_hem[0],lat_hem[1]))
    #set plot settings for each hemisphere
    if lat_hem==(75,25):
        latz = [(75,65),(65,55),(55,45),(45,35),(35,25)]
        legendz = ['65-75 °N','55-65 °N','45-55 °N','35-45 °N','25-35 °N']
        colorz = ['#0066ff','#009900','#e6e600','#ff6600','#ff0000']
        hemisphere = 'Northern Hemisphere'
    else:
        latz = [(-25,-35),(-35,-45),(-45,-55),(-55,-65),(-65,-75)]
        legendz = ['25-35 °S','35-45 °S','45-55 °S','55-65 °S','65-75 °S']
        colorz = ['#0066ff','#009900','#e6e600','#ff6600','#ff0000'][::-1]
        hemisphere = 'Southern Hemisphere'
    #create empty list for binned averages
    mean_lat_lead = []
    #loop over each latitude group
    for lat in latz:
        #create empty list for each bin
        mean_lat_each_lead=[]
        #loop over each lead time (11 in total: 0 to 240 hours with 24 h intervals)
        for lead in leads:   
            #Calculating the mean compensating for cosine latitude weighting
            ds_lat = ds.sel(lat=slice(lat[0],lat[1]),lead=lead)
            #calculate latitude weighted mean average
            ds_lat_mean = my_tools.weighted_average_area(ds_lat)
            #append means for all lead days to latitude bin
            mean_lat_each_lead.append(float(ds_lat_mean))
        #append each latitude bin to get 2D list    
        mean_lat_lead.append(mean_lat_each_lead)
    #convert list to numpy array
    mean_lat_lead = np.array(mean_lat_lead)
    #create plot 
    fig,ax=plt.subplots(figsize=(20,10))
    for i in range(len(mean_lat_lead)):
        ax.plot(mean_lat_lead[i],label=legendz[i],color=colorz[i])
        ax.legend(loc='upper right')
        ax.set_xlabel("Lead time (days)")
        ax.set_ylabel('Mean Error {}'.format(plot_dic[variable]['label']))
        ax.set_ylim(-0.5,0.5)
        ax.set_xticks(np.arange(11))
        ax.grid(b=True,which='major',axis='y')
        ax.margins(0)
    fig.suptitle('GFS {} Mean Error for each Latitude Group compared to ERA5 in the {}'.format(plot_dic[variable]['title'],hemisphere))
    fig.show()
